In [1]:
# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Other tools
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 44.5 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, defaultdict
/usr/local/lib/python3.8/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Load in state of the union address data
import pandas as pd
# Read in data
url = "/content/state_ofthe_union_texts.csv"
df = pd.read_csv(url)

# Subset df to industrial era
industrial = df[(df["Year"] >1877) & (df["Year"] < 1901)]

In [3]:
# Tokenize 

tokenizer = RegexpTokenizer(r'\w+[a-zA-Z]{3,}') # https://stackoverflow.com/questions/46924666/nltk-regexptokenizer-regex-to-retain-just-characters-in-random-text

# create English stop words list
stop_words = stopwords.words('english')
stop_words.extend(['shall', 'applause', 'nation', 'year', 'must', 'world', 'people', 'america', 'american', 'us', 'congress','u', 'y'])

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# compile sample documents into a list
# doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
doc_set = industrial['Text'].values.tolist()

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop_words]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)


# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# filter extremes
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=3, 
                                           id2word = dictionary, 
                                           passes=100)

for idx, topic in ldamodel.print_topics():
  print("Topic: {} \nWords: {}".format(idx, topic ))
  print("\n")
# print(ldamodel.print_topics(num_topics=10, num_words=10))

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

Topic: 0 
Words: 0.005*"nthe" + 0.005*"cancel" + 0.004*"lieuten" + 0.004*"polygami" + 0.004*"scarc" + 0.003*"retain" + 0.003*"doubtless" + 0.003*"mode" + 0.003*"medium" + 0.003*"meridian"


Topic: 1 
Words: 0.006*"pound" + 0.004*"insurg" + 0.004*"circuit" + 0.004*"ounc" + 0.004*"fine" + 0.004*"battl" + 0.004*"partisan" + 0.004*"consum" + 0.004*"ton" + 0.003*"inch"


Topic: 2 
Words: 0.007*"puerto" + 0.006*"rico" + 0.006*"manila" + 0.005*"elector" + 0.005*"governor" + 0.005*"insurg" + 0.005*"kill" + 0.005*"cuban" + 0.005*"choic" + 0.005*"volunt"




In [5]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.4995927947165981


In [4]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.092762 -0.030125       1        1  37.636982
1     -0.077786 -0.051673       2        1  37.124980
0     -0.014976  0.081798       3        1  25.238037, topic_info=          Term       Freq      Total Category  logprob  loglift
779     manila  32.000000  32.000000  Default  30.0000  30.0000
789       nthe  20.000000  20.000000  Default  29.0000  29.0000
1008    puerto  42.000000  42.000000  Default  28.0000  28.0000
558   governor  32.000000  32.000000  Default  27.0000  27.0000
806      pound  36.000000  36.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
219    transit  10.427880  23.853421   Topic3  -5.8924   0.5494
668     travel   9.640265  19.181466   Topic3  -5.9710   0.6888
178       rich   9.587556  19.216050   Topic3  -5.9765   0.6815
1159   struggl  10.073187  32.228897   Topic3  -5.9270   0.2138
332       fish   9.584670  21.996834   Topic3  -5.9768   0.5461

[201 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
685       1  0.084223    abolit
685       2  0.252670    abolit
685       3  0.673787    abolit
687       1  0.183177  accident
687       3  0.732707  accident
...     ...       ...       ...
1266      1  0.082602   workmen
1266      2  0.826023   workmen
872       1  0.859468     wound
872       2  0.057298     wound
872       3  0.057298     wound

[373 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])